In [1]:
# let's use logistic regression to see what features are important


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk.tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
nltk.download('wordnet')
tknzr = TweetTokenizer()
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
import sys
nltk.download('punkt')
!{sys.executable} -m pip install emoji
import emoji

#Data Preprocessing and Feature Engineering

from textblob import TextBlob
import re
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

#from spellchecker import SpellChecker
#!{sys.executable} -m pip install spellchecker

%matplotlib inline

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/perry.oconnor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/perry.oconnor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
train = pd.read_csv("./data/raw/train.csv", encoding='utf8')
test = pd.read_csv("./data/raw/test.csv", encoding='utf8')
sub_sample = pd.read_csv("./data/raw/submission_set.csv", encoding='utf8')

train_disaster = pd.read_csv("./data/raw/train.csv", encoding='utf8')
train_disaster.drop(train_disaster[(train_disaster.disaster != 1)].index, axis=0,inplace=True)

train_nondisaster = pd.read_csv("./data/raw/train.csv", encoding='utf8')
train_nondisaster.drop(train_nondisaster[(train_nondisaster.disaster != 0)].index, axis=0,inplace=True)

In [4]:
class tweet_processor:
    def clean_tweets(self, df):
        df_temp = df.copy()
        df_temp['clean_text'] = df_temp['text'].apply(self.remove_mentions)
        df_temp['clean_text'] = df_temp['clean_text'].apply(lambda x: x.lower())
        df_temp['clean_text'] = df_temp['clean_text'].apply(self.remove_mentions)
        df_temp['clean_text'] = df_temp['clean_text'].apply(self.remove_punctuations)
        df_temp['clean_text'] = df_temp['clean_text'].apply(self.remove_urls)

        return df_temp        
    
    def text_has_emoji(self, text):
        for character in text:
            if character in emoji.UNICODE_EMOJI:
                return True
        return False

    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)

    def strip_emoji(self, text):
        return RE_EMOJI.sub(r'',text)

    # print(strip_emoji('hello🙄🤔'))

    def remove_punctuations(self, text):
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')
        return text

    def remove_digits(self, text):
        for digit in string.digits:
            text = text.replace(digit, '')
        return text

    def remove_mentions(self, text):
        word_list = text.split()
        for word in word_list:
            if word[0] == "@":
                text = text.replace(word,'')
        return text
    # string = "@soundcloud can you recover"
    # print(remove_mentions(string))

    def remove_urls(self, text):
        word_list = text.split()
        for word in word_list:
            if "http" in word:
                text = text.replace(word,'')
        return text

    def contains_proper_nouns(self, text):
        word_list = text.split()
        word_list2 = word_list[1:]
        for word in word_list2:
            is_proper_noun = re.match(r'^[A-Z]',word)
            if is_proper_noun:
                return True

    # string = "hello this is a Test for Perry"
    # print(num_proper_nouns(string))

    def has_lengthening(self, text):
        pattern = re.compile(r"(.)\1{2,}")
        text2 = pattern.sub(r"\1\1",text)
        if text != text2:
            return True
        else:
            return False
        
    # FEATURES
    
    def features(self, df):
        df_temp = df.copy()
        df_temp = self.contains_punctuation(df_temp)
        df_temp = self.contains_multi_sentence(df_temp)
        df_temp = self.contains_mention(df_temp)
        df_temp = self.contains_disaster_word_clean(df_temp)
        df_temp = self.contains_numbers(df_temp)
        df_temp = self.contains_emoji(df_temp)
        df_temp = self.contains_proper_noun(df_temp)
        df_temp = self.has_lengthened_words(df_temp)
        df_temp = self.contains_url(df_temp)
        
        return df_temp

    
    def contains_punctuation(self,df):
        df_temp = df.copy()
        nmbr_of_punct_marks = df_temp['text'].apply(lambda x: len(set(x) & set(string.punctuation)))
        df_temp['feature_has_punctuation'] = nmbr_of_punct_marks.astype(bool).astype(int)
        return df_temp

    def contains_multi_sentence(self,df):
        df_temp = df.copy()
        df_temp['clean_text'] = df_temp['text'].apply(self.remove_urls)
        sentence_markers = [".","!","?"]
        nmbr_of_sentences = df_temp['clean_text'].apply(lambda x: len(set(x) & set(sentence_markers)))
        df_temp['feature_has_multi_sentence'] = nmbr_of_sentences.astype(bool).astype(int)
        return df_temp

    def contains_mention(self, df):
        df_temp = df.copy()
        nmbr_of_mentions = df_temp['text'].apply(lambda x: len(set(x) & set("@")))
        df_temp['feature_has_mention'] = nmbr_of_mentions.astype(bool).astype(int)
        return df_temp

#     def tweet_contains_disaster_word(self, df):
#         df_temp = df.copy()
#         disaster_words = ["flood","hurricane","tornado","fire"]
#         nmbr_of_disasters = df_temp['text'].apply(lambda x: len(set(x.split()) & set(disaster_words)))
#         df_temp['feature_has_disaster_word'] = nmbr_of_disasters.astype(bool).astype(int)
#         return df_temp

    def contains_disaster_word_clean(self, df):
        df_temp = df.copy()
        df_temp = self.clean_tweets(df_temp)
        disaster_words = ["flood","hurricane","tornado","fire","earthquake","cyclone"]
        nmbr_of_disasters = df_temp['clean_text'].apply(lambda x: len(set(x.split()) & set(disaster_words)))
        df_temp['feature_has_disaster_word'] = nmbr_of_disasters.astype(bool).astype(int)
        return df_temp

    def contains_numbers(self, df):
        df_temp = df.copy()
        nmbr_of_numbers = df_temp['text'].apply(lambda x: len(set(x) & set(string.digits)))
        df_temp['feature_has_numbers'] = nmbr_of_numbers.astype(bool).astype(int)
        return df_temp

    def contains_emoji(self, df):
        df_temp = df.copy()
        nmbr_of_emojis = df_temp['text'].apply(self.text_has_emoji)
        df_temp['feature_has_emoji'] = nmbr_of_emojis.astype(bool).astype(int)
        return df_temp

    def contains_proper_noun(self, df):
        df_temp = df.copy()
        df_temp['clean_text'] = df_temp['text'].apply(self.remove_mentions)
        proper_nouns = df_temp['clean_text'].apply(self.contains_proper_nouns)
        df_temp['feature_has_proper_nouns'] = proper_nouns.astype(bool).astype(int)
        return df_temp

    def has_lengthened_words(self, df):
        df_temp = df.copy()
        df_temp['clean_text'] = df_temp['text'].apply(self.remove_punctuations)
        df_temp['clean_text'] = df_temp['clean_text'].apply(lambda x: x.lower())

        df_temp['clean_text'] = df_temp['clean_text'].apply(self.remove_digits)
        df_temp['clean_text'] = df_temp['clean_text'].apply(self.remove_mentions)

        has_lengthened_words = df_temp['clean_text'].apply(self.has_lengthening)
        df_temp['feature_has_lengthened_words'] = has_lengthened_words.astype(bool).astype(int)
        return df_temp

    def contains_url(self, df):
        df_temp = df.copy()
        nmbr_of_urls = df_temp['text'].apply(lambda x: len(re.findall("http",x)))
        df_temp['feature_has_url'] = nmbr_of_urls.astype(bool).astype(int)
        return df_temp

In [5]:
processor = tweet_processor()
df_features = processor.features(train)

In [6]:
df_features.head()

,tweetid,text,disaster_type,disaster,Unnamed: 4,feature_has_punctuation,clean_text,feature_has_multi_sentence,feature_has_mention,feature_has_disaster_word,feature_has_numbers,feature_has_emoji,feature_has_proper_nouns,feature_has_lengthened_words,feature_has_url
0,10001,@TheEllenShow Please check into Salt River hor...,NaN,0,NaN,1,theellenshow please check into salt river hors...,1,1,0,1,0,1,0,0
1,10002,"As for the hurricane, it's already category 1 ...",hurricane,1,NaN,1,as for the hurricane its already category and...,1,0,1,1,0,1,0,0
2,10003,So it looks like my @SoundCloud profile shall ...,NaN,0,NaN,1,so it looks like my soundcloud profile shall b...,1,1,0,0,0,1,0,0
3,10004,@SushmaSwaraj Am sure background check of the ...,NaN,0,NaN,1,sushmaswaraj am sure background check of the c...,1,1,0,0,0,0,0,0
4,10005,Open forex detonation indicator is irretrievab...,NaN,0,NaN,1,open forex detonation indicator is irretrievab...,0,0,0,1,0,1,0,1


In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class contains_url(BaseEstimator, TransformerMixin):
    """Takes in dataframe"""

    def __init__(self):
        pass

    def contains_url(self, text):
        nmbr_of_urls = len(re.findall("http",text))
        return nmbr_of_urls.astype(bool).astype(int)

    def transform(self, df, y=None):
        """The workhorse of this feature extractor"""
        return df['feature_contains_url'].apply(self.contains_url)

    def fit(self, df, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self
    


In [8]:
# build the pipeline
ppl = Pipeline([
              ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='char')),
              ('clf',   ComplementNB())
      ])

pipeline = Pipeline([
    ('feats', FeatureUnion([
        ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='char')), # can pass in either a pipeline
        ('ave', contains_url()) # or a transformer
    ])),
    ('clf', ComplementNB())  # classifier
])

msg_train2, msg_test2, label_train2, label_test2 = train_test_split(train['text'], 
                                                                    train['disaster'], 
                                                                    test_size=0.2, 
                                                                    random_state=30, 
                                                                    stratify=train['disaster'])

# train the classifier
model = ppl.fit(msg_train2,label_train2)

# test the classifier
y_test = model.predict(msg_test2)

print(classification_report(y_test,label_test2))
print(confusion_matrix(y_test,label_test2))
print(accuracy_score(y_test,label_test2))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      2331
           1       0.83      0.99      0.91       473

    accuracy                           0.97      2804
   macro avg       0.92      0.98      0.94      2804
weighted avg       0.97      0.97      0.97      2804

[[2237   94]
 [   3  470]]
0.9654065620542083


In [9]:
msg_train2, msg_test2, label_train2, label_test2 = train_test_split(train['text'], train['disaster'], test_size=0.2, random_state=30, stratify=train['disaster'])
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(msg_train2)

In [10]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [12]:
# homemade_pipeline(msg_train2)

In [13]:
def homemade_pipeline(data):
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(data)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    return X_train_tfidf

clf = ComplementNB().fit(homemade_pipeline(msg_train2), label_train2)
predicted = clf.predict(homemade_pipeline(msg_test2))

ValueError: dimension mismatch

In [14]:
homemade_pipeline(msg_train2)

<11216x28046 sparse matrix of type '<class 'numpy.float64'>'
	with 161249 stored elements in Compressed Sparse Row format>

In [15]:

pipeline.fit(msg_train2,label_train2)
predictions2 = pipeline2.predict(msg_test2)
print(classification_report(predictions2,label_test2))
print(confusion_matrix(predictions2,label_test2))
print(accuracy_score(predictions2,label_test2))

KeyError: 'feature_contains_url'